<a href="https://colab.research.google.com/github/alvarogutyerrez/alvarogutyerrez/blob/master/S10ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Networks: Beating the CAPTCHA?

Qué dirías si te digo que entrenando una red neuronal, podemos hacer que lea el cógido de abajo como si fueran efectivamente números y no una serie de caracteres. Bueno, deberían haberte caído del asiento porque es verdad (En caso contrario, deberías comenzar a replantearte qué es asombroso y lo que no).

In [0]:
    
#@@@@@ ..@.. @@@@@ @@@@@ @...@ @@@@@ @@@@@ @@@@@ @@@@@ @@@@@
#@...@ ..@.. ....@ ....@ @...@ @.... @.... ....@ @...@ @...@
#@...@ ..@.. @@@@@ @@@@@ @@@@@ @@@@@ @@@@@ ....@ @@@@@ @@@@@
#@...@ ..@.. @.... ....@ ....@ ....@ @...@ ....@ @...@ ....@
#@@@@@ ..@.. @@@@@ @@@@@ ....@ @@@@@ @@@@@ ....@ @@@@@ @@@@@        

## Generando la data para entrenar a la bestia

Traspasaremos lo que está arriba al lenguaje más primitivo de todos, los ceros y unos.

In [0]:
import numpy as np
import pandas as pd
import math
import copy
from numpy import array, vectorize, dot
import numpy as np
import math

from numpy import array, vectorize, dot

##GENERANDO LA DATA

digit_0 = [1,1,1,1,1,
           1,0,0,0,1,
           1,0,0,0,1,
           1,0,0,0,1,
           1,1,1,1,1]
            
digit_1 = [1,0,0,0,0,
           1,0,0,0,0,
           1,0,0,0,0,
           1,0,0,0,0,
           1,0,0,0,0]
             
digit_2 = [1,1,1,1,1,
           0,0,0,0,1,
           1,1,1,1,1,
           1,0,0,0,1,
           1,1,1,1,1]

digit_3 = [1,1,1,1,1,
           0,0,0,0,1,
           1,1,1,1,1,
           0,0,0,0,1,
           1,1,1,1,1]

digit_4 = [1,0,0,0,1,
           1,0,0,0,1,
           1,1,1,1,1,
           0,0,0,0,1,
           0,0,0,0,1]
              
digit_5 = [1,1,1,1,1,
           1,0,0,0,0,
           1,1,1,1,1,
           0,0,0,0,1,
           1,1,1,1,1]
              
              
digit_6 = [1,1,1,1,1,
           1,0,0,0,0,
           1,1,1,1,1,
           1,0,0,0,1,
           1,1,1,1,1]
  
digit_7 = [1,1,1,1,1,
           0,0,0,0,1,
           0,0,0,0,1,
           0,0,0,0,1,
           0,0,0,0,1]
  
digit_8 = [1,1,1,1,1,
          1,0,0,0,1,
          1,1,1,1,1,
          1,0,0,0,1,
          1,1,1,1,1]
                          
 
digit_9 = [1,1,1,1,1,
           1,0,0,0,1,
           1,1,1,1,1,
           0,0,0,0,1,
           1,1,1,1,1]   


'''Generating vector X of training'''
inputs = []
for j in range(0,10):
    inputs.append(vars()['digit_' + str(j)] )   

## Ingredientes básicos: Azucas, flores y muchos colores...

Gran parte de la estructura de las ANN se basa en herramientas simples que luego, al combinarse, parecen algo extremadamente complejo. Justamente por esto, vamos por partes...

In [0]:
def step_function(x):
    return 1 if x >= 0 else 0 

     
def perceptron_output(weights, bias, x):
    """returns 1 if the perceptron 'fires', 0 if not"""
    calculation = dot(weights, x) + bias
    return step_function(calculation)
    
def sigmoid(t):
    return 1 / (1 + math.exp(-t))
    
    
def neuron_output(weights, inputs):
    return sigmoid(dot(weights, inputs))
    

#Un paso más adelante: feed_forward

In [0]:
def feed_forward(neural_network, input_vector):
    """takes in a neural network
    (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""
    outputs = []
    # process one layer at a time
    for layer in neural_network:
        input_with_bias = input_vector + [1] # add a bias input
        output = [neuron_output(neuron, input_with_bias) # compute the output
        for neuron in layer] # for each neuron
        outputs.append(output) # and remember it
        # then the input to the next layer is the output of this one
        input_vector = output
    return outputs    


# El ingrediente secreto: Backpropagate

In [0]:
def backpropagate(network, input_vector, targets):
    hidden_outputs, outputs = feed_forward(network, input_vector)
    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target)
        for output, target in zip(outputs, targets)]
    # adjust weights for output layer, one neuron at a time
    for i, output_neuron in enumerate(network[-1]):
        # focus on the ith output layer neuron
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            # adjust the jth weight based on both
            # this neuron's delta and its jth input
            output_neuron[j] -= output_deltas[i] * hidden_output
    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) *dot(output_deltas, [n[i] for n in output_layer])
                for i, hidden_output in enumerate(hidden_outputs)]
    # adjust weights for hidden layer, one neuron at a time
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input

In [0]:
'''Generating Vector Y of training '''              
targets = [[1 if i == j else 0 for i in range(10)]
            for j in range(10)]           

#Inicio de la preparación

In [0]:
'''Settting up initial values '''
np.random.seed(0) # to get repeatable results (seed as ussual)
input_size = 25 # each input is a vector of length 25 (Lenght of training set)
num_hidden = 5 # we'll have 5 neurons in the hidden layer 
output_size = 10 # we need 10 outputs for each input (number to 0 to nine)   



'''Initializing with random numbers for: '''

'''Hidden_Layers (5)'''
# each hidden neuron has one weight per input, plus a bias weight            
hidden_layer = [[np.random.random() for __ in range(input_size + 1)]
                 for __ in range(num_hidden)]                
'''Output_Layers (10)'''
# each output neuron has one weight per hidden neuron, plus a bias weight
output_layer = [[np.random.random() for __ in range(num_hidden + 1)]
                 for __ in range(output_size)]     
                 
# As the book mention, the network is a list of (layers) of  
#list (neurons) of list (weights). And it starts out with random weights 
#The ones tha we just generate randomnly.
network = [hidden_layer, output_layer]                              


#Entrenando al Monstruo

In [0]:
for __ in range(10000):
    for input_vector, target_vector in zip(inputs, targets):
        backpropagate(network, input_vector, target_vector)                 
                 

#Generando Predicciones

In [0]:
''' Generar funcion de prediccion'''
def predict(input):
    return feed_forward(network, input)[-1]   

##Easy peasy: ¿Cómo luce un 7?

In [19]:
prediccion_7=pd.DataFrame(predict(inputs[7])).round(3) 
print(prediccion_7)

       0
0  0.000
1  0.000
2  0.059
3  0.359
4  0.137
5  0.000
6  0.000
7  0.885
8  0.041
9  0.049


##El sucio tres

In [0]:
the_three=[0,1,1,1,0,  # .@@@.
           0,0,0,1,1,  # ...@@
           0,0,1,1,0,  # ..@@.
           0,0,0,1,1,  # ...@@
           0,1,1,1,0]  # .@@@

In [24]:
the_three_predicho=pd.DataFrame(predict(the_three)).round(3)               
print(the_three_predicho)  
#Not that bad!

       0
0  0.000
1  0.000
2  0.045
3  0.675
4  0.058
5  0.000
6  0.000
7  0.000
8  0.068
9  0.197


#Sklearn: Aplicando la librería favorita de la casa.

In [25]:
''' Ocupando Skilearn '''
from sklearn.neural_network import MLPClassifier
weird_three = pd.DataFrame([0,1,1,1,0, # .@@@.
               0,0,0,1,1, # ...@@
               0,0,1,1,0, # ..@@.
               0,0,0,1,1, # ...@@
               0,1,1,1,0]) # .@@@. 
weird_three = weird_three.T
                           
clf_1 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5), random_state=1)
Y_train = pd.DataFrame(targets)
X_train = pd.DataFrame(inputs)

clf_1.fit( X_train,Y_train)                         

prediccion_weird3 =pd.DataFrame(clf_1.predict(weird_three)).round(3)
print(prediccion_weird3)

   0  1  2  3  4  5  6  7  8  9
0  0  0  0  1  1  1  0  0  0  1


In [26]:

digit_7 = pd.DataFrame([1,1,1,1,1,
              0,0,0,0,1,
              0,0,0,0,1,
              0,0,0,0,1,
              0,0,0,0,1]).T
  

prediccion_7even=pd.DataFrame(clf_1.predict(digit_7)).round(3)
print(prediccion_7even)

   0  1  2  3  4  5  6  7  8  9
0  0  0  0  0  0  0  0  0  0  0


#Sklean, pero haciendolo bien!

## El regreso del sucio tres

In [39]:
'''Otro formato de explicada '''

clf_2 = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5), random_state=1)

Y_train2 = pd.DataFrame([0,1,2,3,4,5,6,7,8,9])
clf_2.fit(X_train,Y_train2 )                         

prediccion_weird3 =pd.DataFrame(clf_2.predict(weird_three)).round(3)

print(prediccion_weird3)
#aparentemente sklearn lo hizo peor que from scratch...........

   0
0  8


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


##Peace of cake?

In [31]:
prediccion_7even_2 =pd.DataFrame(clf_2.predict(digit_7)).round(3)
print(prediccion_7even_2)
#awesome!

   0
0  7
